In [1]:
import csv

import pandas as pd
import polars as pl

In [2]:
df = pd.read_csv("customers-2000000/customers-2000000.csv")

# Most obvious

In [ ]:
chunk_df = pd.read_csv('customers-2000000/customers-2000000.csv', chunksize=10000)

In [ ]:
last_name_counts = {}

for chunk in chunk_df:
    last_name_count = chunk['Last Name'].value_counts()
    for last_name, count in last_name_count.items():
        if last_name not in last_name_counts:
            last_name_counts[last_name] = count
        else:
            last_name_counts[last_name] += count

last_name_counts = sorted(last_name_counts.items(), key=lambda x: x[1], reverse=True)
print(last_name_counts)

[('Bird', 2133), ('Gill', 2133), ('Hubbard', 2126), ('Lara', 2119), ('Lamb', 2115), ('Craig', 2109), ('Blevins', 2109), ('Becker', 2107), ('Porter', 2106), ('Hardy', 2106), ('Cantrell', 2105), ('Hodge', 2105), ('Weber', 2104), ('Phelps', 2104), ('Page', 2101), ('Garrett', 2101), ('Rocha', 2101), ('Macdonald', 2100), ('Kennedy', 2097), ('Montgomery', 2095), ('Klein', 2095), ('Pacheco', 2093), ('Shea', 2091), ('Mcguire', 2091), ('Arellano', 2088), ('Bernard', 2087), ('Graves', 2086), ('Riggs', 2086), ('Choi', 2086), ('Bryan', 2086), ('Stevenson', 2086), ('Durham', 2085), ('Huber', 2084), ('Moran', 2083), ('Villa', 2083), ('Preston', 2083), ('Wilcox', 2082), ('Wise', 2082), ('Odonnell', 2082), ('Riley', 2082), ('Farrell', 2080), ('Mercer', 2079), ('Summers', 2079), ('Sanchez', 2079), ('Dudley', 2078), ('Cowan', 2078), ('Kelley', 2078), ('Vazquez', 2077), ('Walker', 2077), ('Barton', 2076), ('Salinas', 2076), ('Norman', 2076), ('Marks', 2076), ('Curry', 2076), ('Dougherty', 2075), ('Juarez

# Tired of Pandas

In [6]:
lazy_df = pl.scan_csv("customers-2000000/customers-2000000.csv")

In [10]:
last_name_counts = lazy_df.group_by("Last Name").agg(pl.count("Last Name").alias("count"))

print(last_name_counts.collect().sort(by="count", descending=True))

shape: (1_000, 2)
┌───────────┬───────┐
│ Last Name ┆ count │
│ ---       ┆ ---   │
│ str       ┆ u32   │
╞═══════════╪═══════╡
│ Gill      ┆ 2133  │
│ Bird      ┆ 2133  │
│ Hubbard   ┆ 2126  │
│ Lara      ┆ 2119  │
│ Lamb      ┆ 2115  │
│ …         ┆ …     │
│ Castillo  ┆ 1881  │
│ Novak     ┆ 1877  │
│ Black     ┆ 1877  │
│ Abbott    ┆ 1872  │
│ Wilson    ┆ 1865  │
└───────────┴───────┘


# By Force

In [12]:
def read_row_by_row(file_path: str):
    with open(file_path, newline='', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        for row in reader:
            yield row

last_name_counts = {}
for row in read_row_by_row("customers-2000000/customers-2000000.csv"):
    last_name = row.get("Last Name", None)
    if last_name:
        if last_name not in last_name_counts:
            last_name_counts[last_name] = 1
        else:
            last_name_counts[last_name] += 1

last_name_counts = sorted(last_name_counts.items(), key=lambda x: x[1], reverse=True)
print(last_name_counts)

[('Bird', 2133), ('Gill', 2133), ('Hubbard', 2126), ('Lara', 2119), ('Lamb', 2115), ('Blevins', 2109), ('Craig', 2109), ('Becker', 2107), ('Porter', 2106), ('Hardy', 2106), ('Hodge', 2105), ('Cantrell', 2105), ('Phelps', 2104), ('Weber', 2104), ('Rocha', 2101), ('Page', 2101), ('Garrett', 2101), ('Macdonald', 2100), ('Kennedy', 2097), ('Montgomery', 2095), ('Klein', 2095), ('Pacheco', 2093), ('Shea', 2091), ('Mcguire', 2091), ('Arellano', 2088), ('Bernard', 2087), ('Riggs', 2086), ('Choi', 2086), ('Graves', 2086), ('Bryan', 2086), ('Stevenson', 2086), ('Durham', 2085), ('Huber', 2084), ('Moran', 2083), ('Preston', 2083), ('Villa', 2083), ('Wilcox', 2082), ('Odonnell', 2082), ('Riley', 2082), ('Wise', 2082), ('Farrell', 2080), ('Sanchez', 2079), ('Summers', 2079), ('Mercer', 2079), ('Dudley', 2078), ('Kelley', 2078), ('Cowan', 2078), ('Walker', 2077), ('Vazquez', 2077), ('Norman', 2076), ('Marks', 2076), ('Barton', 2076), ('Curry', 2076), ('Salinas', 2076), ('Juarez', 2075), ('Dougherty